In [71]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import xgboost as xgb
# import lightgbm as lgb
import warnings
warnings.filterwarnings(action= 'ignore')
import torch
import pdb

In [72]:
ROOT_DIR = os.path.abspath('/home/ayhaos/Daily Research Progress/night/interpretation-of-animal-actions-in-the-video-stre')
TRAIN_CSV_PATH = os.path.join(ROOT_DIR, 'x_train.csv')
TARGET_CSV_PATH = os.path.join(ROOT_DIR, 'y_train.csv')
TEST_CSV_PATH = os.path.join(ROOT_DIR, 'x_test.csv')
SAMPLE_SUB_PATH = os.path.join(ROOT_DIR, 'sample_submission.csv')

In [73]:
df = pd.read_csv(TRAIN_CSV_PATH)
y_df = pd.read_csv(TARGET_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)
df.head(10)

,id,cat,cow,dog,horse,sheep,bounds_x,bounds_y,bounds_width,bounds_height,...,L_F_paw_y,R_F_paw_v,R_F_paw_x,R_F_paw_y,L_B_paw_v,L_B_paw_x,L_B_paw_y,R_B_paw_v,R_B_paw_x,R_B_paw_y
0,265,0,1,0,0,0,37,16,240,170,...,167,1,161,168,1,94,173,1,55,177
1,676,0,0,0,1,0,43,8,208,236,...,213,1,193,209,1,77,232,1,115,233
2,441,0,0,1,0,0,65,0,234,222,...,0,0,0,0,0,0,0,0,0,0
3,328,0,1,0,0,0,15,8,272,178,...,171,1,190,172,1,52,176,1,35,178
4,102,1,0,0,0,0,2,1,148,146,...,130,1,68,131,1,3,128,0,0,0
5,766,0,0,0,1,0,1,0,296,266,...,246,0,0,0,1,288,253,1,250,248
6,120,1,0,0,0,0,21,0,276,184,...,0,0,0,0,0,0,0,0,0,0
7,926,0,0,0,0,1,211,137,164,246,...,368,1,290,367,0,0,0,1,224,355
8,914,0,0,0,0,1,45,97,671,649,...,641,1,218,621,1,584,638,1,652,639
9,649,0,0,0,1,0,79,15,214,170,...,175,1,118,173,0,0,0,0,0,0


In [74]:
test_df.head(10)
df.columns


Index(['id', 'cat', 'cow', 'dog', 'horse', 'sheep', 'bounds_x', 'bounds_y',
       'bounds_width', 'bounds_height', 'L_eye_v', 'L_eye_x', 'L_eye_y',
       'R_eye_v', 'R_eye_x', 'R_eye_y', 'L_ear_v', 'L_ear_x', 'L_ear_y',
       'R_ear_v', 'R_ear_x', 'R_ear_y', 'Nose_v', 'Nose_x', 'Nose_y',
       'Throat_v', 'Throat_x', 'Throat_y', 'Tail_v', 'Tail_x', 'Tail_y',
       'withers_v', 'withers_x', 'withers_y', 'L_F_elbow_v', 'L_F_elbow_x',
       'L_F_elbow_y', 'R_F_elbow_v', 'R_F_elbow_x', 'R_F_elbow_y',
       'L_B_elbow_v', 'L_B_elbow_x', 'L_B_elbow_y', 'R_B_elbow_v',
       'R_B_elbow_x', 'R_B_elbow_y', 'L_F_knee_v', 'L_F_knee_x', 'L_F_knee_y',
       'R_F_knee_v', 'R_F_knee_x', 'R_F_knee_y', 'L_B_knee_v', 'L_B_knee_x',
       'L_B_knee_y', 'R_B_knee_v', 'R_B_knee_x', 'R_B_knee_y', 'L_F_paw_v',
       'L_F_paw_x', 'L_F_paw_y', 'R_F_paw_v', 'R_F_paw_x', 'R_F_paw_y',
       'L_B_paw_v', 'L_B_paw_x', 'L_B_paw_y', 'R_B_paw_v', 'R_B_paw_x',
       'R_B_paw_y'],
      dtype='object')

In [75]:
df.describe()

,id,cat,cow,dog,horse,sheep,bounds_x,bounds_y,bounds_width,bounds_height,...,L_F_paw_y,R_F_paw_v,R_F_paw_x,R_F_paw_y,L_B_paw_v,L_B_paw_x,L_B_paw_y,R_B_paw_v,R_B_paw_x,R_B_paw_y
count,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,...,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000
mean,478.068047,0.193787,0.204142,0.205621,0.205621,0.190828,40.448225,25.292899,239.211538,215.662722,...,153.023669,0.677515,102.707101,148.744083,0.596154,103.616864,126.184911,0.578402,78.893491,123.331361
std,278.844496,0.395557,0.403372,0.404454,0.404454,0.393245,45.914110,29.659900,83.348094,72.999818,...,114.070073,0.467774,92.661854,119.018746,0.491031,117.579513,116.089671,0.494180,100.760136,118.386722
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,78.000000,102.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,233.750000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,7.000000,194.000000,170.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,484.500000,0.000000,0.000000,0.000000,0.000000,0.000000,26.500000,16.000000,236.000000,200.000000,...,177.000000,1.000000,107.000000,176.000000,1.000000,73.000000,164.000000,1.000000,43.500000,160.000000
75%,720.500000,0.000000,0.000000,0.000000,0.000000,0.000000,53.000000,34.000000,272.000000,246.000000,...,218.000000,1.000000,167.000000,218.000000,1.000000,204.250000,205.250000,1.000000,130.250000,206.000000
max,964.000000,1.000000,1.000000,1.000000,1.000000,1.000000,347.000000,276.000000,896.000000,662.000000,...,641.000000,1.000000,618.000000,621.000000,1.000000,755.000000,638.000000,1.000000,652.000000,639.000000


In [76]:
# merged_df = pd.merge(df, y_df, on="id")

# merged_df[-5:][-5:]

In [77]:
feature_type = 1

In [78]:
df.isna().sum()

if feature_type == 1:
# If removing _v and id values
    binary_col = []
    target_cols = []
    for col in df.columns:
        if '_v' in col:
            binary_col.append(col)
    print(binary_col)
    one_hot_cols = df.columns[1:6]
    useless_cols = binary_col
    print(useless_cols)
    


['L_eye_v', 'R_eye_v', 'L_ear_v', 'R_ear_v', 'Nose_v', 'Throat_v', 'Tail_v', 'withers_v', 'L_F_elbow_v', 'R_F_elbow_v', 'L_B_elbow_v', 'R_B_elbow_v', 'L_F_knee_v', 'R_F_knee_v', 'L_B_knee_v', 'R_B_knee_v', 'L_F_paw_v', 'R_F_paw_v', 'L_B_paw_v', 'R_B_paw_v']
['L_eye_v', 'R_eye_v', 'L_ear_v', 'R_ear_v', 'Nose_v', 'Throat_v', 'Tail_v', 'withers_v', 'L_F_elbow_v', 'R_F_elbow_v', 'L_B_elbow_v', 'R_B_elbow_v', 'L_F_knee_v', 'R_F_knee_v', 'L_B_knee_v', 'R_B_knee_v', 'L_F_paw_v', 'R_F_paw_v', 'L_B_paw_v', 'R_B_paw_v']


In [79]:
useful_cols = [col for col in df.columns[1:] if col not in useless_cols ] #id, _v removed 
# print(useful_cols)

test_df = test_df[useful_cols]
X, y = df[useful_cols], np.asarray(y_df.iloc[:,1:])

Y = []
'''
0: run
1: go
2: lie
3: sit
4: stand
5: nothing
'''
for one_h in y:
    for ind,val in enumerate(one_h):
        if val == 1:
            Y.append(ind)
            continue
    # print(Y)

df['label'] = Y
# df.drop('f_27', axis=1, inplace=True)

In [87]:
norm = True

if norm == True:
    scaler = StandardScaler().fit(X.iloc[:,5:])
    X_transformed = scaler.transform(X.iloc[:,5:])
    norm_cols = [col for col in useful_cols if col not in one_hot_cols]
    df[norm_cols] = X_transformed
else:
    X_transformed = X

In [88]:
df.head()

,id,cat,cow,dog,horse,sheep,bounds_x,bounds_y,bounds_width,bounds_height,...,R_F_paw_v,R_F_paw_x,R_F_paw_y,L_B_paw_v,L_B_paw_x,L_B_paw_y,R_B_paw_v,R_B_paw_x,R_B_paw_y,label
0,265,0,1,0,0,0,-0.075157,-0.313547,0.009467,-0.625981,...,1,0.629559,0.161909,1,-0.081851,0.403565,1,-0.237308,0.453669,4
1,676,0,0,0,1,0,0.055618,-0.583471,-0.374749,0.278800,...,1,0.975156,0.506647,1,-0.226541,0.912169,1,0.358607,0.927045,4
2,441,0,0,1,0,0,0.535129,-0.853396,-0.062574,0.086877,...,0,-1.109228,-1.250679,0,-0.881902,-1.087766,0,-0.783563,-1.042538,0
3,328,0,1,0,0,0,-0.554668,-0.583471,0.393683,-0.516311,...,1,0.942756,0.195542,1,-0.439320,0.429426,1,-0.435946,0.462122,1
4,102,1,0,0,0,0,-0.838015,-0.819655,-1.095155,-0.954993,...,1,-0.374834,-0.149197,1,-0.856368,0.015647,0,-0.783563,-1.042538,3


In [89]:
df[useful_cols]

,cat,cow,dog,horse,sheep,bounds_x,bounds_y,bounds_width,bounds_height,L_eye_x,...,R_B_knee_x,R_B_knee_y,L_F_paw_x,L_F_paw_y,R_F_paw_x,R_F_paw_y,L_B_paw_x,L_B_paw_y,R_B_paw_x,R_B_paw_y
0,0,1,0,0,0,-0.075157,-0.313547,0.009467,-0.625981,-1.241946,...,-0.287489,0.192649,0.870907,0.122615,0.629559,0.161909,-0.081851,0.403565,-0.237308,0.453669
1,0,0,0,1,0,0.055618,-0.583471,-0.374749,0.278800,-1.241946,...,0.341197,0.759764,0.461065,0.526174,0.975156,0.506647,-0.226541,0.912169,0.358607,0.927045
2,0,0,1,0,0,0.535129,-0.853396,-0.062574,0.086877,0.405138,...,-0.768826,-1.049136,-1.208292,-1.342482,-1.109228,-1.250679,-0.881902,-1.087766,-0.783563,-1.042538
3,0,1,0,0,0,-0.554668,-0.583471,0.393683,-0.516311,-1.241946,...,-0.375898,0.309984,0.481057,0.157707,0.942756,0.195542,-0.439320,0.429426,-0.435946,0.462122
4,1,0,0,0,0,-0.838015,-0.819655,-1.095155,-0.954993,-0.006633,...,-0.533069,-0.002907,0.081211,-0.201987,-0.374834,-0.149197,-0.856368,0.015647,-0.783563,-1.042538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,0,1,0,0,0,-0.554668,-0.819655,0.321643,-0.379223,-1.241946,...,-0.375898,0.720653,-1.208292,-1.342482,-1.109228,-1.250679,-0.881902,-1.087766,-0.783563,-1.042538
672,0,0,0,0,1,1.799292,1.575922,5.556589,4.994630,5.785616,...,1.225286,3.673560,5.329190,4.052933,5.565121,3.937217,0.582021,4.075860,1.431253,4.105429
673,0,0,0,0,1,0.360761,-0.448509,-0.758966,-0.543729,0.780308,...,0.105440,0.349095,0.760950,0.148934,0.737558,0.220767,-0.201007,0.394945,0.279151,0.445216
674,1,0,0,0,0,-0.467484,-0.583471,0.465724,-0.022794,-0.464156,...,1.343164,0.016648,0.700973,0.570039,0.294761,0.590730,1.586341,-0.174002,1.798733,-0.019707


Logistic Regression

In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_transformed, Y, test_size=0.4, random_state=42)


In [90]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [91]:
predictions = model.predict(X_valid)
accuracy = model.score(X_valid, y_valid)
print("accruacy: ", accuracy)

accruacy:  0.6420664206642066


XGBOOST

In [59]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indices, valid_indices) in enumerate(kf.split(X= df[useful_cols], y=Y)):
    df.loc[valid_indices, "kfold"] = int(fold)

In [69]:
final_preds = []
 
for fold in range(5):
    df_train = df[df['kfold'] != fold].reset_index(drop=True)
    df_valid = df[df['kfold'] == fold].reset_index(drop=True)
    
    X_train, y_train = df_train[useful_cols], df_train['label']
    X_valid, y_valid = df_valid[useful_cols], df_valid['label']
    X_test = test_df
    
    # eval_set = [(X_train, y_train), (X_valid, y_valid)]
    eval_set = [(X_valid, y_valid)]
    

    model = xgb.XGBClassifier(
        objective='multi:softprob',
        learning_rate=0.0003,
        n_estimators=1000,
        max_depth=4,
        eval_metric=["error"],
        tree_method='gpu_hist',
        gpu_id=0
    )
    model.fit(X_train, y_train, verbose=False)

    train_preds = model.predict_proba(X_train)[:,1]
    valid_preds = model.predict_proba(X_valid)[:,1]
    test_preds = model.predict_proba(X_test)[:,1]
    final_preds.append(test_preds)
    
    # print(f'Fold: {fold}, Train AuC: {accuracy_score(train_preds, y_train)}, Valid AuC: {accuracy_score(valid_preds, y_valid)}')

In [70]:
train_pred = model.predict(X_train)
valid_pred = model.predict(X_valid)
test_pred = model.predict(X_test)
print(accuracy_score(train_pred,y_train))
print(accuracy_score(valid_pred,y_valid))

0.8096118299445472
0.6222222222222222


# ensemble xgboost logistic regression

In [93]:
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
# Initialize logistic regression and XGBoost models
log_reg = LogisticRegression()
xgb = XGBClassifier()

# Create ensemble using VotingClassifier
ensemble = VotingClassifier(estimators=[('log_reg', log_reg), ('xgb', xgb)], voting='soft')

# Train the ensemble model
ensemble.fit(X_train, y_train)

# Predictions on the test set
predictions = ensemble.predict(X_valid)

# Evaluate the model
accuracy = ensemble.score(X_valid, y_valid)
print("Accuracy:", accuracy)

Accuracy: 0.6642066420664207
